In [1]:
!mlflow --version

mlflow, version 2.14.2


Q1: mlflow, version 2.14.2

In [2]:
dest_path='homework_models'
raw_data_path='https://d37ci6vzurychx.cloudfront.net/trip-data/'

!mkdir -p $dest_path
!ls -l

итого 256
drwxrwxr-x 2 nikto nikto   4096 июл 29 17:33 artifacts_local
-rw-rw-r-- 1 nikto nikto   5601 июл 29 14:44 homework.ipynb
drwxrwxr-x 2 nikto nikto   4096 июл 27 14:53 homework_models
-rw-r--r-- 1 nikto nikto 225280 июл 29 14:43 mlflow.db
drwxrwxr-x 4 nikto nikto   4096 июл 29 17:31 mlruns
-rw-rw-r-- 1 nikto nikto   2510 июл 27 13:06 preprocess_data.py
drwxrwxr-x 2 nikto nikto   4096 июл 29 14:43 __pycache__
-rw-rw-r-- 1 nikto nikto   1225 июл 29 17:31 train.py


import preprocess_data
<br>- it isn't needed, because the function run_data_prep doesn't work with import because of 'click' context.
It would work if the function was splited to 2, the 1st for import, the 2nd for cli, like:

def run_data_prep(..):..

@click..

dev run_data_prep_cli(..): run_data_prep(..)`


In [3]:
# doesn't work because there's 'click' context
# preprocess_data.run_data_prep(raw_data_path=raw_data_path,dest_path=dest_path)
! python preprocess_data.py --raw_data_path="$raw_data_path" --dest_path "$dest_path"
# ! python preprocess_data.py --help

Q2: 4

so I took the liberty of changing train.py as it was mentioned above for preprocess_data.py, now it works both with import and with cli.

! python train.py --help

There's no need to wrap in with with mlflow.start_run(), mlflow.autolog() is triggered automaticly.

In [4]:
from train import run_train
# rt=run_train(dest_path)
rt=run_train(dest_path)
print(rt)

2024/07/29 17:42:11 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/07/29 17:42:11 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/07/29 17:42:11 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ae6cf6662b0245bc868be07102a1f0cc', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/07/29 17:42:11 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


(RandomForestRegressor(max_depth=10, random_state=0), array([30.31046662, 21.65971103, 24.30442126, ..., 19.72563812,
       21.65971103, 14.82687034]), 5.431162180141208)


/home/nikto/study/mlops-zoomcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


mlflow ui --backend-store-uri sqlite:///mlflow.db

Q3: 2

mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts_local

In [6]:
if callable(run_train): del run_train
from train import run_train
rt_s=run_train(dest_path,'http://127.0.0.1:5000')
print(rt_s)

2024/07/29 17:43:38 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/07/29 17:43:38 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/07/29 17:43:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ea5e4a2daf55453a8fdb0f4c1b114b73', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/07/29 17:43:38 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


(RandomForestRegressor(max_depth=10, random_state=0), array([30.31046662, 21.65971103, 24.30442126, ..., 19.72563812,
       21.65971103, 14.82687034]), 5.431162180141208)


/home/nikto/study/mlops-zoomcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
Q4: default-artifact-root